# 4.6 API経由でDiscoveryを使う

## 4.6.1 APIの初期化

In [ ]:
# 必要ライブラリの導入
!pip install -U ibm_watson | tail -n 1

In [ ]:
# リスト4.6.1
# 資格情報の設定 (個別に設定します)

discovery_credentials = {
  "apikey": "xxxx",
  "iam_apikey_description": "xxxx",
  "iam_apikey_name": "xxxx",
  "iam_role_crn": "xxxx",
  "iam_serviceid_crn": "xxxx",
  "url": "xxxx"
}

# Discovery APIの初期化

import json
import os
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

version = '2019-04-30'

authenticator = IAMAuthenticator(discovery_credentials['apikey'])
discovery = DiscoveryV1(
    version=version,
    authenticator=authenticator
)
discovery.set_service_url(discovery_credentials['url'])

In [ ]:
# リスト 4.6.2
# environment_id、collection_id、configuration_id の取得
# すでにUIで1つのprivate collectionが作成済みであることが前提

# environment id の取得
environments = discovery.list_environments().get_result()['environments']
environment_id = environments[0]['environment_id']
if environment_id == 'system':
    environment_id = environments[1]['environment_id']
print('environment_id: ', environment_id)

# collection id の取得
collection_id = discovery.list_collections(environment_id ).get_result()['collections'][0]['collection_id']
print('collection_id: ' , collection_id)

# configuration idの取得
configuration_id = discovery.list_configurations(environment_id).get_result()['configurations'][0]['configuration_id']
print('configuration_id: ', configuration_id)

## 4.6.2 文書のロードと削除

In [ ]:
#  リスト 4.6.3 
# 文書ロード関数
# collection_id: 対象コレクション
# sample_data: 書き込み対象テキスト (json形式の配列)
# key_name: 文書のユニークキー名称

def load_text( collection_id, sample_data, key_name):
    for item in sample_data:
        
        # item毎にワークのjsonファイルを作成
        print(item)
        key = item.get(key_name)
        filename = str(key) + '.json'
        f = open(filename, 'w')
        json.dump(item, f)
        f.close()
        
        # 書き込み可能かのチェック
        collection = discovery.get_collection(environment_id, collection_id).get_result()
        proc_docs = collection['document_counts']['processing']
        while True:
            if proc_docs < 20:
                break
            print('busy. waiting..')
            time.sleep(10)
            collection = discovery.get_collection(environment_id, collection_id)
            proc_docs = collection['document_counts']['processing']

        # jsonファイル名を引数にDiscoveryへデータロード
        with open(filename) as f:
            add_doc = discovery.add_document(environment_id, collection_id, file = f)
        os.remove(filename)

In [ ]:
#  リスト 4.6.4 
# 文書ロードサンプル

# ロードテスト用テキスト
sample_data = [
    {'app_id': 1, 'title': '最初のテキスト', 'text': 'サンプルテキストその1。'},
    {'app_id': 2, 'title': '2番目のテキスト', 'text': '新幹線はやぶさが好きです。'},
    {'app_id': 3, 'title': '3番目のテキスト', 'text': '令和元年に転職しました。'},
]

# 文書ロードテスト
load_text(collection_id, sample_data, 'app_id')

In [ ]:
# リスト4.6.5
# 特定のコレクションの全文書を削除する関数
# collection_id: 対象コレクション

def delete_all_docs(collection_id):

    # 文書件数取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection['document_counts']['available']

    results = discovery.query(environment_id, collection_id, return_fields='id', count=doc_count).get_result()["results"]
    ids = [item["id"] for item in results]

    for id in ids:
        print('deleting doc: id =' + id)
        discovery.delete_document(environment_id, collection_id, id)

In [ ]:
# リスト4.6.6

# 全件削除テスト
delete_all_docs(collection_id)

## 4.6.3 検索

In [ ]:
# リスト4.6.7

# 検索用関数
# collection_id: 検索対象コレクション
# query_text: 検索条件式
# return_fields: 出力項目

def query_documents(collection_id, query_text, return_fields):
    # 文書件数取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection['document_counts']['available']
    print('doc_count: ', doc_count)

    query_results = discovery.query(environment_id, collection_id, 
        query=query_text, 
        count=doc_count, 
        return_fields=return_fields).get_result()[ "results"]
    return query_results

### textフィールドに「サンプル」を含む文書の検索

In [ ]:
# リスト4.6.8
#「サンプル」をキーとした検索 

query_text = 'text:サンプル'
return_fields = 'app_id,title,text'
query_results = query_documents(collection_id, query_text, return_fields)

print(json.dumps(query_results, indent=2, ensure_ascii=False))

### textフィールドに「はやぶさ」を含む文書の検索
同じ方式で「はやぶさ」を含む文字列の検索を試します。

In [ ]:
# リスト4.6.9 
# はやぶさ」をキーとした検索

query_text = 'text:はやぶさ'
return_fields = 'app_id,title,text'
query_results = query_documents(collection_id, query_text, return_fields)

print(json.dumps(query_results, indent=2, ensure_ascii=False))

## 4.6.4 形態素辞書の利用

In [ ]:
# リスト 4.6.10
# 形態素辞書の定義例

custom_list = [
    {
        "text":"はやぶさ",
        "tokens":["はやぶさ"],
        "readings":[ "ハヤブサ"],
        "part_of_speech":"カスタム名詞"
    }
  ]

In [ ]:
# リスト4.6.11
# 形態素辞書の登録用関数

def register_tokenization_dictionary(collection_id,  tokenization_rules):
    res = discovery.create_tokenization_dictionary(environment_id, collection_id, tokenization_rules=tokenization_rules)
    import time
    res = discovery.get_tokenization_dictionary_status(environment_id, collection_id).get_result()
    while res['status'] == 'pending':
        time.sleep(10)
        res = discovery.get_tokenization_dictionary_status(environment_id, collection_id).get_result()
        print(res)    

In [ ]:
# リスト4.6.12
# 形態素辞書登録用関数の呼び出し例

register_tokenization_dictionary(collection_id,  custom_list)

In [ ]:
# リスト 4.6.13
# 「はやぶさ」で検索できることの確認

delete_all_docs(collection_id)
load_text(collection_id, sample_data, 'app_id')

import time
time.sleep(30)

query_text = 'text:はやぶさ'
return_fields = 'app_id,title,text'
query_results = query_documents(collection_id, query_text, return_fields)

print(json.dumps(query_results, indent=2, ensure_ascii=False))